# W281 Final Project: Intel Image Classification Model #

In [20]:
pip install scikit-learn

     ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
     --------------------------------------- 0.0/11.0 MB 640.0 kB/s eta 0:00:18
     --------------------------------------- 0.0/11.0 MB 640.0 kB/s eta 0:00:18
     --------------------------------------- 0.1/11.0 MB 871.5 kB/s eta 0:00:13
      --------------------------------------- 0.2/11.0 MB 1.1 MB/s eta 0:00:10
      --------------------------------------- 0.2/11.0 MB 1.1 MB/s eta 0:00:10
      --------------------------------------- 0.2/11.0 MB 1.1 MB/s eta 0:00:10
     - -------------------------------------- 0.5/11.0 MB 1.6 MB/s eta 0:00:07
     --- ------------------------------------ 0.8/11.0 MB 2.3 MB/s eta 0:00:05
     ---- ----------------------------------- 1.2/11.0 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.2/11.0 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.2/11.0 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/11.0 M


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter

from PIL import Image
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,BatchNormalization,MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
def load_images(directory, num_files=5):
    images = []
    file_list = os.listdir(directory)[:num_files]  # Load only the first 5 files

    for filename in tqdm(file_list, desc=f"Loading images from {directory}"):
        img_path = os.path.join(directory, filename)
        img = Image.open(img_path)
        img = img.resize((150, 150))  # Resize image to 150 x 150
        images.append(img)

    return images

In [5]:
# Image Paths

buildings_train = './seg_train/buildings'
forest_train = './seg_train/forest'
glacier_train = './seg_train/glacier'
mountain_train = './seg_train/mountain'
sea_train = './seg_train/sea'
street_train = './seg_train/street'

buildings_test = './seg_test/buildings'
forest_test = './seg_test/forest'
glacier_test = './seg_test/glacier'
mountain_test = './seg_test/mountain'
sea_test = './seg_test/sea'
street_test = './seg_test/street'

In [6]:
# Load images from each category
buildings_img = load_images(buildings_train)
forest_img = load_images(forest_train)
glacier_img = load_images(glacier_train)
mountain_img = load_images(mountain_train)
sea_img = load_images(sea_train)
street_img = load_images(street_train)

Loading images from ./seg_train/street: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [7]:
buildings_img

[<PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>]

## Load Data ##

In [8]:
def load_data(datasets):
    
    output = []

    for dataset in datasets:
        images, labels = [], []
        print(f"Loading {dataset}...")
        i = 0
        for folder in os.listdir(dataset):
            label = i # Converting word labels to int (i.e. buildings = 0)
            i = i+1
            folder_path = os.path.join(dataset, folder)

            for file in tqdm(os.listdir(folder_path), desc=f"Processing {folder}"):
                img_path = os.path.join(folder_path, file)

                image = Image.open(img_path).resize((150, 150))
                images.append(image)
                labels.append(label)
                
        images = np.stack(images)
        labels = np.array(labels, dtype='int32')
        output.append((images, labels))
        
    return output

In [16]:
datasets = ["C:\\Users\\PeterBenzoni\\repo\\w281_Fall2024_FinalProject_Brown_Benzoni_Olaya\\seg_train", 
        "C:\\Users\\PeterBenzoni\\repo\w281_Fall2024_FinalProject_Brown_Benzoni_Olaya\\seg_train"]


train_path, test_path = datasets

In [ ]:


(train_images, train_labels), (test_images, test_labels) = load_data(datasets)


In [10]:
# Normalization
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [11]:
# Training Data Distribution
unique, counts = np.unique(train_labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 2191, 1: 2271, 2: 2404, 3: 2512, 4: 2274, 5: 2382}


In [12]:
avg = sum(counts)/len(unique)
print(avg)

2339.0


In [13]:
# Test Data Distribution
test_unique, test_counts = np.unique(test_labels, return_counts=True)
print(dict(zip(test_unique, test_counts)))

{0: 2191, 1: 2271, 2: 2404, 3: 2512, 4: 2274, 5: 2382}


In [14]:
test_avg = sum(test_counts)/len(test_unique)
print(test_avg)

2339.0


In [18]:

# Prepare data using ImageDataGenerator
data_gen = ImageDataGenerator(rescale=1./255)

train_generator = data_gen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = data_gen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Extract data for RandomForest Classifier
train_images, train_labels = [], []
for i in range(len(train_generator)):
    x, y = train_generator[i]
    train_images.extend(x)
    train_labels.extend(y)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Preprocess the labels
le = LabelEncoder()
train_labels_rf = le.fit_transform(np.argmax(train_labels, axis=1))

# Split the training data for Random Forest
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(train_images.reshape(len(train_images), -1), train_labels_rf, test_size=0.2, random_state=42)

# RandomForest Model
print("Training Random Forest Classifier...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_rf, y_train_rf)

# Evaluate RandomForest
y_pred_rf = rf.predict(X_test_rf)
print("Random Forest Classification Report:")
print(classification_report(y_test_rf, y_pred_rf))
print("Accuracy:", accuracy_score(y_test_rf, y_pred_rf))

# Define a simple CNN Model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Compile the CNN model
cnn_model = create_cnn_model((150, 150, 3), train_generator.num_classes)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN Model
cnn_model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the CNN Model
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_generator)
print(f"CNN Model Accuracy: {cnn_accuracy * 100:.2f}%")

# ResNet50 Transfer Learning Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

resnet_model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the ResNet model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ResNet Model
resnet_model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the ResNet Model
resnet_loss, resnet_accuracy = resnet_model.evaluate(test_generator)
print(f"ResNet50 Model Accuracy: {resnet_accuracy * 100:.2f}%")

# Summary of Model Performances
print("\nModel Performance Summary:")
print(f"Random Forest Model Accuracy: {accuracy_score(y_test_rf, y_pred_rf) * 100:.2f}%")
print(f"CNN Model Accuracy: {cnn_accuracy * 100:.2f}%")
print(f"ResNet50 Model Accuracy: {resnet_accuracy * 100:.2f}%")


Found 14034 images belonging to 6 classes.
Found 14034 images belonging to 6 classes.
Training Random Forest Classifier...
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.40      0.46       436
           1       0.72      0.85      0.78       453
           2       0.54      0.59      0.56       475
           3       0.62      0.68      0.65       535
           4       0.51      0.41      0.45       477
           5       0.61      0.64      0.63       431

    accuracy                           0.60      2807
   macro avg       0.59      0.59      0.59      2807
weighted avg       0.59      0.60      0.59      2807

Accuracy: 0.5963662272889205


c:\Users\PeterBenzoni\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 126s 284ms/step - accuracy: 0.5504 - loss: 1.1309 - val_accuracy: 0.7448 - val_loss: 0.6924
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 137s 312ms/step - accuracy: 0.7867 - loss: 0.6023 - val_accuracy: 0.8137 - val_loss: 0.5020
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 113s 259ms/step - accuracy: 0.8491 - loss: 0.4130 - val_accuracy: 0.9127 - val_loss: 0.2723
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 109s 249ms/step - accuracy: 0.9002 - loss: 0.2800 - val_accuracy: 0.9461 - val_loss: 0.1760
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 111s 252ms/step - accuracy: 0.9420 - loss: 0.1628 - val_accuracy: 0.9528 - val_loss: 0.1378
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 119s 270ms/step - accuracy: 0.9692 - loss: 0.1013 - val_accuracy: 0.9803 - val_loss: 0.0712
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 122s 277ms/step - accuracy: 0.9806 - loss: 0.0628 - val_accuracy: 0.9803 - val_loss: 0.0591
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 121s 275ms/step - accuracy: 0.9807 -